# Osnovni vzorec AutoGen

V tem vzorcu kode boste uporabili [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework za ustvarjanje osnovnega agenta.

Cilj tega vzorca je prikazati korake, ki jih bomo kasneje uporabili v dodatnih vzorcih kode pri implementaciji različnih vzorcev agentov.


## Uvoz potrebnih Pythonovih paketov


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Ustvarite odjemalca

V tem primeru bomo uporabili [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) za dostop do LLM.

`model` je definiran kot `gpt-4o-mini`. Poskusite spremeniti model v drug model, ki je na voljo na tržnici GitHub Models, da vidite različne rezultate.

Za hiter preizkus bomo izvedli preprost poziv - `Kaj je glavno mesto Francije`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiranje agenta

Zdaj, ko smo nastavili `client` in potrdili, da deluje, ustvarimo `AssistantAgent`. Vsakemu agentu lahko dodelimo:
**ime** - Kratko ime, ki bo uporabno za sklicevanje nanj v večagentnih tokovih.
**model_client** - Odjemalec, ki ste ga ustvarili v prejšnjem koraku.
**orodja** - Razpoložljiva orodja, ki jih agent lahko uporabi za dokončanje naloge.
**sistemsko sporočilo** - Metapoziv, ki določa nalogo, vedenje in ton LLM-ja.

Sistemsko sporočilo lahko spremenite, da preverite, kako LLM odgovarja. O `orodjih` bomo govorili v Lekciji #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Zaženi agenta

Spodnja funkcija bo zagnala agenta. Uporabimo metodo `on_message`, da posodobimo stanje agenta z novim sporočilom.

V tem primeru posodobimo stanje z novim sporočilom uporabnika, ki je `"Načrtuj mi odličen sončen dopust"`.

Vsebino sporočila lahko spremenite, da vidite, kako LLM drugače odgovarja.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
